In [114]:
import numpy as np
import json, re, os, requests, uuid, base64, io
from openai import AzureOpenAI
import logging
from azure.storage.blob import BlobServiceClient
import tempfile
from io import BytesIO
from PIL import Image
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor, as_completed
import ast
import requests
from azure.search.documents import SearchClient
from azure.core.pipeline.transport import RequestsTransport
from azure.core.credentials import AzureKeyCredential


def gpt_client(prompt, contents = []):
    try:
        client = AzureOpenAI(
            azure_endpoint="https://hrtransformation.openai.azure.com/",
            api_key="AFd7UMq9pERgXT9w1z7IOvcD84px0dHEUHwHgMh1PDycGm6JJVC5JQQJ99BBAC77bzfXJ3w3AAABACOG28lv",
            api_version="2024-05-01-preview"
        )


        completion = client.chat.completions.create(
            model="gpt-5-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant who strictly follows user instructions."},
                {"role": "user", "content": [{"type":"text", "text": prompt}, *contents]}
            ],
            response_format={"type": "json_object"},
            timeout=300
        )

        response=completion.choices[0].message.content
        
        logging.info(f"gpt_client - Response received successfully.")
        return response
    except Exception as e:
        response = {}
        logging.info(f"gpt_client - LLM not responding due to error: {e}")
        return response

In [115]:
query = "What is Section 80C and what are the guidelines related to it? Are there any latest notifications regarding it?"

In [116]:
query_class = gpt_client(f"""
    Classify the query on basis of the context. The categories for classification are [Act, Rule, Circular]. 
    A query can have multiple categories based on context.
    
    Query: {query}
    Return a flat JSON object which includes the list of categories as follows:
    {{
        "category": <one or more of the above listed categories as applicable>
    }}
""")

logging.info(f"categories: {query_class}")
query_class = json.loads(query_class)


In [117]:
print(query_class)

{'category': ['Act', 'Rule', 'Circular']}


In [ ]:
def search_index(search_keywords, index_container, top_k=20):
    """
    Perform a search using Azure Cognitive Search.
    """
    # Azure Search parameters
    SEARCH_ENDPOINT = "https://hrtransformation.search.windows.net"
    SEARCH_API_KEY = "kFixV6YiahYG6AvOIInbQQfqCJrXbN0IUWTlw9ZYkOAzSeDFroKy"
    INDEX_NAME = index_container

    logging.info(f"Searching for policies using keywords: {search_keywords}")
    headers = {
        'Content-Type': 'application/json',
        'api-key': SEARCH_API_KEY
    }
    url = f"{SEARCH_ENDPOINT}/indexes/{INDEX_NAME}/docs/search?api-version=2025-11-01-preview"
    data = {
        "search": search_keywords,
        "top": top_k
    }
    response = requests.post(url, headers=headers, json=data, verify=False)

    if response.status_code == 200:
        chunks = response.json()
        return json.dumps(chunks, indent=5)
    else:
        logging.error(f"Azure Search error: {response.status_code}")
        return 'error'


In [133]:
updatedQuery = gpt_client(f'''
    Rewrite the user query to maximize retrieval relevance:
    User query: {query}
''')

chunks_list = {}
cat_list = ast.literal_eval(str(query_class.get("category")))
print(cat_list)
for cat in cat_list:
    print(cat)
    if cat == "Act": index_container = "taxation-acts"; 
    elif cat == "Rule": index_container = "taxation-rules"; 
    elif cat == "Circular": index_container = "taxation-circular-notification"
    chunks = search_index(updatedQuery, index_container, top_k=10)
    print(chunks)
chunks_list.extend(json.loads(chunks))

['Act', 'Rule', 'Circular']
Act


c:\Users\37279\AppData\Local\anaconda3\envs\taxation\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'hrtransformation.search.windows.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
ERROR:root:Azure Search error: 400


<Response [400]>
error
Rule


c:\Users\37279\AppData\Local\anaconda3\envs\taxation\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'hrtransformation.search.windows.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
ERROR:root:Azure Search error: 400


<Response [400]>
error
Circular


c:\Users\37279\AppData\Local\anaconda3\envs\taxation\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'hrtransformation.search.windows.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
ERROR:root:Azure Search error: 400


<Response [400]>
error


AttributeError: 'dict' object has no attribute 'extend'

In [31]:
print(chunks_list)

{}
